In [2]:
%run data.py

data_path: C:\Users\Pop\Documents\GitHub\utcc_independent_study\training\..\data\data.xlsx
model_dir: C:\Users\Pop\Documents\GitHub\utcc_independent_study\training\..\model
K=1: train_1: 800 | test_1: 200
K=2: train_2: 800 | test_2: 200
K=3: train_3: 800 | test_3: 200
K=4: train_4: 800 | test_4: 200
K=5: train_5: 800 | test_5: 200
Validation หูฟังไร้สาย: val_1: 41
Validation คีย์บอร์ดและเมาส์: val_2: 34
Validation ลำโพงบลูทูธ: val_3: 37
Validation สมาร์ทวอร์ช: val_4: 40
Validation เครื่องฟอกอากาศ: val_5: 35


In [3]:
train = 'train_' + '5'
test = 'test_' + '5'
train_data = eval(train)
test_data = eval(test)

gradient_accumulation_steps = 4 # Set up gradient accumulation
batch_size = 16  # Reduced batch size
num_epochs = 5

In [3]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_absolute_error
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import time

model_name = "airesearch/wangchanberta-base-att-spm-uncased"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1 # regression
model1 = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer1 = AutoTokenizer.from_pretrained(model_name)

loss_fn = torch.nn.L1Loss() # MAE loss
device = torch.device("cpu") # Move your model to the device (CPU)

model = model1
tokenizer = tokenizer1

model.to(device)

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Tokenize data and create dataloader
encoded_data = tokenizer.batch_encode_plus(
    train_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=100,
    return_tensors='pt'
)

labels = torch.tensor(train_data['score'].values, dtype=torch.float32)  # Convert labels to float32
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Preprocess the test data
encoded_test_data = tokenizer.batch_encode_plus(
    test_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

test_labels = torch.tensor(test_data['score'].values, dtype=torch.float32)
test_dataset = TensorDataset(encoded_test_data['input_ids'], encoded_test_data['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Fine-tune the model

early_stopping_loss = float('inf')

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_dataloader:
        step += 1
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if step % gradient_accumulation_steps == 0:
            optimizer.step()
            epoch_loss += loss.item()
    
    if epoch_loss/step >= early_stopping_loss:
        break
        
    early_stopping_loss = epoch_loss/step

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / step}')

end_time = time.time()
elapsed_time = end_time - start_time

print(f'training time: {elapsed_time}')

# Evaluate the model's predictions against the actual values
model.eval()
predictions = []
actuals = []

start_time = time.time()

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = logits.squeeze(1).cpu().numpy()

        predictions.extend(preds)
        actuals.extend(labels.cpu().numpy())
        
end_time = time.time()
elapsed_time = end_time - start_time

print(f'test time: {elapsed_time}')

mae = mean_absolute_error(actuals, predictions)
print(f'Mean Absolute Error: {mae:.6f}')


result_df = pd.DataFrame({'actual': actuals, 'predicted': predictions})
result_df['comment'] = test_data['comment']
result_df = result_df[['comment', 'actual', 'predicted']].sort_values(by='predicted', ascending=False)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

Epoch: 1, Loss: 0.5668996810913086
Epoch: 2, Loss: 0.17273732781410217
training time: 1575.9777982234955
test time: 70.7753438949585
Mean Absolute Error: 1.169225


In [4]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_absolute_error
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import time

model_name = "pythainlp/thainer-corpus-v2-base-model"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1 # regression
model1 = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer1 = AutoTokenizer.from_pretrained(model_name)

loss_fn = torch.nn.L1Loss() # MAE loss
device = torch.device("cpu") # Move your model to the device (CPU)

model = model1
tokenizer = tokenizer1

model.to(device)

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Tokenize data and create dataloader
encoded_data = tokenizer.batch_encode_plus(
    train_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=100,
    return_tensors='pt'
)

labels = torch.tensor(train_data['score'].values, dtype=torch.float32)  # Convert labels to float32
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Preprocess the test data
encoded_test_data = tokenizer.batch_encode_plus(
    test_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

test_labels = torch.tensor(test_data['score'].values, dtype=torch.float32)
test_dataset = TensorDataset(encoded_test_data['input_ids'], encoded_test_data['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Fine-tune the model

early_stopping_loss = float('inf')

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_dataloader:
        step += 1
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if step % gradient_accumulation_steps == 0:
            optimizer.step()
            epoch_loss += loss.item()
    
    if epoch_loss/step >= early_stopping_loss:
        break
        
    early_stopping_loss = epoch_loss/step

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / step}')

end_time = time.time()
elapsed_time = end_time - start_time

print(f'training time: {elapsed_time}')

# Evaluate the model's predictions against the actual values
model.eval()
predictions = []
actuals = []

start_time = time.time()

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = logits.squeeze(1).cpu().numpy()

        predictions.extend(preds)
        actuals.extend(labels.cpu().numpy())
        
end_time = time.time()
elapsed_time = end_time - start_time

print(f'test time: {elapsed_time}')

mae = mean_absolute_error(actuals, predictions)
print(f'Mean Absolute Error: {mae:.6f}')


result_df = pd.DataFrame({'actual': actuals, 'predicted': predictions})
result_df['comment'] = test_data['comment']
result_df = result_df[['comment', 'actual', 'predicted']].sort_values(by='predicted', ascending=False)

Some weights of the model checkpoint at pythainlp/thainer-corpus-v2-base-model were not used when initializing CamembertForSequenceClassification: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at pythainlp/thainer-corpus-v2-base-model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this 

Epoch: 1, Loss: 0.4198994755744934
Epoch: 2, Loss: 0.21395529806613922
Epoch: 3, Loss: 0.13011595249176025
Epoch: 4, Loss: 0.10111084938049317
Epoch: 5, Loss: 0.08293647333979606
training time: 2456.9514729976654
test time: 53.9320068359375
Mean Absolute Error: 1.062453


In [4]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_absolute_error
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import time

model_name = "KoichiYasuoka/roberta-base-thai-spm-upos"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1 # regression
model1 = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer1 = AutoTokenizer.from_pretrained(model_name)

loss_fn = torch.nn.L1Loss() # MAE loss
device = torch.device("cpu") # Move your model to the device (CPU)

model = model1
tokenizer = tokenizer1

model.to(device)

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Tokenize data and create dataloader
encoded_data = tokenizer.batch_encode_plus(
    train_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=100,
    return_tensors='pt'
)

labels = torch.tensor(train_data['score'].values, dtype=torch.float32)  # Convert labels to float32
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Preprocess the test data
encoded_test_data = tokenizer.batch_encode_plus(
    test_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

test_labels = torch.tensor(test_data['score'].values, dtype=torch.float32)
test_dataset = TensorDataset(encoded_test_data['input_ids'], encoded_test_data['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Fine-tune the model

early_stopping_loss = float('inf')

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_dataloader:
        step += 1
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if step % gradient_accumulation_steps == 0:
            optimizer.step()
            epoch_loss += loss.item()
    
    if epoch_loss/step >= early_stopping_loss:
        break
        
    early_stopping_loss = epoch_loss/step

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / step}')

end_time = time.time()
elapsed_time = end_time - start_time

print(f'training time: {elapsed_time}')

# Evaluate the model's predictions against the actual values
model.eval()
predictions = []
actuals = []

start_time = time.time()

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = logits.squeeze(1).cpu().numpy()

        predictions.extend(preds)
        actuals.extend(labels.cpu().numpy())
        
end_time = time.time()
elapsed_time = end_time - start_time

print(f'test time: {elapsed_time}')

mae = mean_absolute_error(actuals, predictions)
print(f'Mean Absolute Error: {mae:.6f}')


result_df = pd.DataFrame({'actual': actuals, 'predicted': predictions})
result_df['comment'] = test_data['comment']
result_df = result_df[['comment', 'actual', 'predicted']].sort_values(by='predicted', ascending=False)

Some weights of the model checkpoint at KoichiYasuoka/roberta-base-thai-spm-upos were not used when initializing RobertaForSequenceClassification: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at KoichiYasuoka/roberta-base-thai-spm-upos and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this mode

Epoch: 1, Loss: 0.4588659280538559
Epoch: 2, Loss: 0.20947953283786774
Epoch: 3, Loss: 0.11519215226173402
training time: 3005.6669759750366
test time: 65.89408206939697
Mean Absolute Error: 0.985246


In [4]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_absolute_error
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import time

model_name = "monsoon-nlp/bert-base-thai"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1 # regression
model1 = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer1 = AutoTokenizer.from_pretrained(model_name)

loss_fn = torch.nn.L1Loss() # MAE loss
device = torch.device("cpu") # Move your model to the device (CPU)

model = model1
tokenizer = tokenizer1

model.to(device)

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Tokenize data and create dataloader
encoded_data = tokenizer.batch_encode_plus(
    train_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=100,
    return_tensors='pt'
)

labels = torch.tensor(train_data['score'].values, dtype=torch.float32)  # Convert labels to float32
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Preprocess the test data
encoded_test_data = tokenizer.batch_encode_plus(
    test_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

test_labels = torch.tensor(test_data['score'].values, dtype=torch.float32)
test_dataset = TensorDataset(encoded_test_data['input_ids'], encoded_test_data['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Fine-tune the model

early_stopping_loss = float('inf')

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_dataloader:
        step += 1
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if step % gradient_accumulation_steps == 0:
            optimizer.step()
            epoch_loss += loss.item()
    
    if epoch_loss/step >= early_stopping_loss:
        break
        
    early_stopping_loss = epoch_loss/step

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / step}')

end_time = time.time()
elapsed_time = end_time - start_time

print(f'training time: {elapsed_time}')

# Evaluate the model's predictions against the actual values
model.eval()
predictions = []
actuals = []

start_time = time.time()

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = logits.squeeze(1).cpu().numpy()

        predictions.extend(preds)
        actuals.extend(labels.cpu().numpy())
        
end_time = time.time()
elapsed_time = end_time - start_time

print(f'test time: {elapsed_time}')

mae = mean_absolute_error(actuals, predictions)
print(f'Mean Absolute Error: {mae:.6f}')


result_df = pd.DataFrame({'actual': actuals, 'predicted': predictions})
result_df['comment'] = test_data['comment']
result_df = result_df[['comment', 'actual', 'predicted']].sort_values(by='predicted', ascending=False)

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not

Epoch: 1, Loss: 0.3510856080055237
Epoch: 2, Loss: 0.20380317091941832
Epoch: 3, Loss: 0.1429926210641861
training time: 1712.810251712799
test time: 50.111971378326416
Mean Absolute Error: 1.188243
